<a href="https://colab.research.google.com/github/sushil79g/60daysUdacity/blob/master/PROJECT%3Asentiment/sentiment_udacity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import files
files.upload()

Saving employee_reviews.csv to employee_reviews.csv


In [0]:
#numpy for mathmatical computation
import numpy as np
#pandas for dataframe computation i.e overall sql processing
import pandas as pd

In [0]:
#importing file with pandas
file = pd.read_csv('employee_reviews.csv')
#lets look @dataset
file.head(3)

In [0]:
# lets chop out only important attribute for out project
imp_column = ['pros','cons','overall-ratings','summary']

In [0]:
our_dataset = file[imp_column]

In [0]:
our_dataset.head(5)

In [0]:
our_dataset['all_review'] = our_dataset['pros'] + our_dataset['cons'] + our_dataset['summary']

In [0]:
import re #python module for regular expression
import nltk #python module for natural language processing
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
nltk.download('stopwords')

In [0]:
def remove_punctuation(x):
    line = re.sub(r'[^a-zA-Z ]+', ' ', str(x))
    return line

In [0]:
def remove_stopword(x):
    line = [word for word in x if not word in set(stopwords.words('english'))]
    return line

In [0]:
def conv_lower(x):
    new = []
    for item in x:
        new.append(item.lower())
    return new

In [0]:
nltk.download('wordnet')
lemma = nltk.wordnet.WordNetLemmatizer()
lemma.lemmatize('article')
nltk.download('averaged_perceptron_tagger')

In [0]:
from textblob import TextBlob
def app_lamitz(x):
  try:
    words = x.split()
    result = []
    for wor in words:
      # print(nltk.pos_tag(wor)[0][1][0])
      if nltk.pos_tag(wor)[0][1][0]=='J':
        wor = lemma.lemmatize(wor)
#         print(wor)
      result.append(wor)
    value = ' '.join(result)
    b = TextBlob(value)
    return b.correct()
  except:
    b = TextBlob(x)
    return b.correct()

In [0]:
def preprocess_all_review(x):
    re_punct = remove_punctuation(x)
    re_stopword = remove_stopword(re_punct)
    line_lower = conv_lower(re_stopword)
    result = ' '.join(re_stopword)
    return line_lower
our_dataset['preprocess_review'] = our_dataset['all_review'].apply(preprocess_all_review)

In [0]:
#another approch of doing
def preprocess_all_review(x):
    line = re.sub(r'[^a-zA-Z ]+', ' ', str(x))
    line = line.lower()
    line = line.split()
    line = [word for word in line if not word in set(stopwords.words('english'))]
    line = ' '.join(line)
    return line
our_dataset['preprocess_review'] = our_dataset['all_review'].apply(preprocess_all_review)

In [0]:
our_dataset.head(5)

In [0]:
import seaborn as sns

In [0]:
sns.countplot(x = 'overall-ratings', data=our_dataset)

In [0]:
import fastai
from fastai import *
from fastai.text import *
from functools import partial
import io
import os

In [0]:
file.drop('Unnamed: 0',axis=1,inplace=True)

In [0]:

# file= file.dropna(how='any',axis=0,inplace=True)
file.dropna(axis=0, how='any', inplace=True)
# file.isna()
file.head(3)

In [0]:
dataset = pd.DataFrame()
dataset['text'] = file['pros']
dataset['label'] = 'positive'
dataset.head()

In [0]:
neg_dataset = pd.DataFrame()
neg_dataset['text'] = file['cons']
neg_dataset['label'] = 'negative'

In [0]:
final_dataset = dataset.append(neg_dataset)

In [0]:
final_dataset = final_dataset.reindex_axis(['label','text','is_valid'], axis=1)
final_dataset.head(5)

In [0]:
def preprocess_all_review(x):
    line = re.sub(r'[^a-zA-Z ]+', ' ', str(x))
    line = line.lower()
    line = line.split()
    line = [word for word in line if not word in set(stopwords.words('english'))]
    line = ' '.join(line)
    return line
final_dataset['text']=final_dataset['text'].apply(preprocess_all_review)

In [0]:
final_dataset['is_valid'].unique()

In [0]:
final_dataset['is_valid'] = 'False'
change = final_dataset.sample(27011).index
final_dataset.loc[change,'is_valid'] = 'True'

In [0]:
def polar(x):
  if x>=3.0:
    return 'positive'
  else:
    return 'negative'
file['polarity'] = file['overall-ratings'].apply(polar)

In [0]:
final_dataset.to_csv('texts.csv')
data_lm = TextLMDataBunch.from_csv(csv_name='texts.csv',path='.',text_cols='text',label_cols='label')


In [0]:
data_clas = TextClasDataBunch.from_csv(csv_name='texts.csv',path='.',text_cols='text',label_cols='label', vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
data_lm.save('data_lm_export.pkl')
data_clas.save('data_clas_export.pkl')

In [0]:
data_lm = load_data('.', 'data_lm_export.pkl')
data_clas = load_data('.', 'data_clas_export.pkl', bs=16)

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

In [0]:
learn.predict("This is a review about", n_words=10)

In [0]:
learn.save_encoder('ft_enc')

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_enc')

In [0]:
learn.fit_one_cycle(1, 1e-2)

In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

In [0]:
learn.predict("Great to work here, such a good environment")

In [0]:
learn.predict("i have a mix feeling for this place")

In [0]:
learn.predict('i am extremly overwhelmed by the treacherous and torrid journey to the eutopia where i found the splendid mesmerizing scenic canvas to color my dream')

In [0]:
learn.predict('the girl with sweet smile and charismatic attitude makes my heart full of love for her')